In [1]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install statsmodels
!pip install matplotlib
!pip install feature-engine
!pip install scikit-learn
!pip install xgboost




  Using cached numpy-1.25.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
  Using cached pandas-2.0.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached matplotlib-3.7.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached contourpy-1.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.41.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
  Using cached kiwisolver-1.4.4-cp311-cp311-manylinux_2_1

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import feature_engine
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV




### Hyperparameter Optimization ---- Model is fitted using cross validation. 
#### GridSearchCV ---- Fits multiple models looping through a hyperparameter list over the model. 

In [29]:
data_url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"

# Read the dataset into a DataFrame
df = pd.read_csv(data_url)

# Rename the 'price' column to 'target' in the original DataFrame
df.rename(columns={'medv': 'price'}, inplace=True)
df = df.sample(frac=0.5, random_state=101)

# Display the modified DataFrame
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,price
195,0.01381,80.0,0.46,0,0.422,7.875,32.0,5.6484,4,255,14.4,394.23,2.97,50.0
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
434,13.91340,0.0,18.10,0,0.713,6.208,95.0,2.2222,24,666,20.2,100.63,15.17,11.7
458,7.75223,0.0,18.10,0,0.713,6.301,83.7,2.7831,24,666,20.2,272.21,16.23,14.9
39,0.02763,75.0,2.95,0,0.428,6.595,21.8,5.4011,3,252,18.3,395.63,4.32,30.8


In [30]:
X_train, X_test, y_train, y_test = train_test_split(
                                        df.drop(['price'], axis=1),
                                        df['price'],
                                        test_size=0.2,
                                        random_state=(101))
print("* Train set:", X_train.shape, y_train.shape,"\n* Test set:", X_test.shape, y_test.shape)         



* Train set: (202, 13) (202,) 
* Test set: (51, 13) (51,)


In [31]:
# ML Algorithms 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [36]:
def pipeline_random_forest_reg():
    pipeline = Pipeline([
        ("feature_scaling", StandardScaler()),
        ("feature_selection", SelectFromModel(RandomForestRegressor(random_state=101))),
        ("model", RandomForestRegressor(random_state=101)),
    ])
    return pipeline

In [37]:
param_grid = {"model__n_estimators": [10, 20]}


In [38]:
grid = GridSearchCV(estimator=pipeline_random_forest_reg(),
                    param_grid = param_grid,
                    cv = 2,
                    n_jobs = -2,
                    verbose = 3,
                    scoring = 'r2')

grid.fit(X_train, y_train)                    

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV 1/2] END ............model__n_estimators=10;, score=0.618 total time=   0.3s
[CV 2/2] END ............model__n_estimators=10;, score=0.695 total time=   0.2s
[CV 2/2] END ............model__n_estimators=20;, score=0.675 total time=   0.3s
[CV 1/2] END ............model__n_estimators=20;, score=0.590 total time=   0.3s


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('feature_scaling', StandardScaler()),
                                       ('feature_selection',
                                        SelectFromModel(estimator=RandomForestRegressor(random_state=101))),
                                       ('model',
                                        RandomForestRegressor(random_state=101))]),
             n_jobs=-2, param_grid={'model__n_estimators': [10, 20]},
             scoring='r2', verbose=3)

In [40]:
grid.cv_results_

{'mean_fit_time': array([0.11716878, 0.12686276]),
 'std_fit_time': array([0.00134242, 0.00382948]),
 'mean_score_time': array([0.00876665, 0.00940406]),
 'std_score_time': array([2.95639038e-05, 2.69055367e-04]),
 'param_model__n_estimators': masked_array(data=[10, 20],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'model__n_estimators': 10}, {'model__n_estimators': 20}],
 'split0_test_score': array([0.61830829, 0.5896595 ]),
 'split1_test_score': array([0.6945313 , 0.67511803]),
 'mean_test_score': array([0.6564198 , 0.63238876]),
 'std_test_score': array([0.03811151, 0.04272927]),
 'rank_test_score': array([1, 2], dtype=int32)}

In [39]:

def pipeline_random_forest_reg():
    pipeline = Pipeline([
        ("feature_scaling", StandardScaler()),
        ("feature_selection", SelectFromModel(RandomForestRegressor(random_state=101))),
        ("model", RandomForestRegressor(random_state=101)),
    ])
    return pipeline

# Define the parameter grid with the correct parameter name
param_grid = {"model__n_estimators": [10, 20]}

# Create the pipeline
pipeline = pipeline_random_forest_reg()

# Create the GridSearchCV object
grid = GridSearchCV(estimator=pipeline,
                    param_grid=param_grid,
                    cv=2,
                    n_jobs=-2,
                    verbose=3,
                    scoring='r2')

# Fit the GridSearchCV to perform hyperparameter tuning
grid.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV 2/2] END ............model__n_estimators=20;, score=0.675 total time=   0.1s
[CV 1/2] END ............model__n_estimators=10;, score=0.618 total time=   0.1s
[CV 2/2] END ............model__n_estimators=10;, score=0.695 total time=   0.1s
[CV 1/2] END ............model__n_estimators=20;, score=0.590 total time=   0.1s


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('feature_scaling', StandardScaler()),
                                       ('feature_selection',
                                        SelectFromModel(estimator=RandomForestRegressor(random_state=101))),
                                       ('model',
                                        RandomForestRegressor(random_state=101))]),
             n_jobs=-2, param_grid={'model__n_estimators': [10, 20]},
             scoring='r2', verbose=3)